In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import time
from scipy import io
from scipy.ndimage import (gaussian_filter, median_filter, 
                           binary_erosion, binary_dilation)
#from skimage.morphology import binary_erosion, binary_dilation, disk
from skimage.morphology import disk

# ***1. package definition***

In [2]:
import os
import numpy as np

import astra, tomopy
import tifffile, h5py
import matplotlib.pylab as mpp

def reader(fn, dtype, path=None, dim=None):
    if dtype == 'prj':
        with h5py.File(fn, 'r') as f:
            if dim is None:
                tem = -np.log((f['/img_tomo'][:] - f['/img_dark_avg'][:])/(f['/img_bkg_avg'][:] - f['/img_dark_avg'][:]))
                tem[np.isinf(tem)] = 1
                tem[np.isnan(tem)] = 1
            else:
                tem = -np.log((f['/img_tomo'][dim] - f['/img_dark_avg'][dim])/(f['/img_bkg_avg'][dim] - f['/img_dark_avg'][dim]))
                tem[np.isinf(tem)] = 1
                tem[np.isnan(tem)] = 1
        return tem
    if dtype == 'obj':
        if dim is None:
            return 0
        else:
            return np.zeros(dim)
    if dtype == 'theta':
        if dim is None:
            with h5py.File(fn, 'r') as f:
                if path is None:            
                    tem = f['/angle'][:]*np.pi/180
                else:
                    tem = f[path][:]*np.pi/180
            step = np.partition(np.diff(tem), 5)[4]
            print(f"{step = }")
            was = np.argmax(np.diff(tem))
            print(f"{was = }")
            wan = int((tem[was+1] - tem[was])//step)
            print(f"{wan = }")
            return (np.concatenate((tem[:was], np.linspace(tem[was], tem[was+1], wan, endpoint=False), tem[was+1:])),
                    was, was+wan-1)
        else:
            with h5py.File(fn, 'r') as f:
                if path is None:
                    tem = f['/angle'][:]*np.pi/180
                else:
                    tem = f[path][:]*np.pi/180
            return tem, dim[0], dim[1]
    if dtype == 'obj_supp':
        with h5py.File(fn, 'r') as f:
            if dim is None:
                if path is None:
                    return (f['/obj_supp'][:]>0).astype(np.int8)
                else:
                    return (f[path][:]>0).astype(np.int8)
            else:
                if path is None:
                    return (f['/obj_supp'][dim]>0).astype(np.int8)
                else:
                    return (f[path][dim]>0).astype(np.int8)
    if dtype == 'sin_supp':
        with h5py.File(fn, 'r') as f:
            if dim is None:
                if path is None:
                    return (f['/sin_supp'][:]>0).astype(np.int8)
                else:
                    return (f[path][:]>0).astype(np.int8)
            else:
                if path is None:
                    return (f['/sin_supp'][dim]>0).astype(np.int8)
                else:
                    return (f[path][dim]>0).astype(np.int8)
            
def bprjr(prj, cfg):
    """
    "back projector": prj -> obj
    Inputs:
        alg: str, algorithm name in ['gridrec', 'art', 'fbp', 'bart', 'sirt', 'tv',
                                     'mlem', 'osem', 'ospml_hybrid', 'ospml_quad', 
                                     'pml_hybrid', 'pml_quad', 'grad',
                                     'astra_3D', 'astra_2D']
        data: 'img' and 'theta' for alg in ['gridrec', 'art', 'fbp', 'bart', 'sirt', 'tv',
                                     'mlem', 'osem', 'ospml_hybrid', 'ospml_quad', 
                                     'pml_hybrid', 'pml_quad', 'grad']
              'img' and 'num_iters' for alg in ['astra_2D']
              'num_iters' for alg in ['astra_3D']
              
    """
    alg = cfg['name']
    if alg in ['gridrec', 'art', 'fbp', 'bart', 'sirt', 'tv', 'mlem', 'osem', 
               'ospml_hybrid', 'ospml_quad', 'pml_hybrid', 'pml_quad', 'grad']:
        # print(cfg['cfg'])
        mod_obj = tomopy.recon(prj, cfg['theta'], **cfg['cfg'])
    elif alg in ['astra.3D']:
        algorithm_id = astra.algorithm.create(cfg['cfg']['alg_cfg'])
        astra.algorithm.run(algorithm_id, cfg['cfg']['num_iters'])
        mod_obj = astra.data3d.get(cfg['cfg']['alg_cfg']['ReconstructionDataId'])
        astra.algorithm.delete(algorithm_id)
        astra.data3d.delete(cfg['cfg']['alg_cfg']['ReconstructionDataId'])
        astra.data3d.delete(cfg['cfg']['alg_cfg']['ProjectionDataId'])
        astra.functions.clear()
    elif alg in ['astra.2D']:
        mod_obj = []
        proj_geom = astra.data2d.get_geometry(cfg['cfg']['alg_cfg']['ProjectionDataId'])
        for ii in range(prj.shape[0]):
            cfg['cfg']['alg_cfg']['ProjectionDataId'] = astra.data2d.create('-sino', proj_geom, prj[ii])
            algorithm_id = astra.algorithm.create(cfg['cfg']['alg_cfg'])
            astra.algorithm.run(algorithm_id, cfg['cfg']['num_iters'])
            mod_obj.append(astra.data2d.get(cfg['cfg']['alg_cfg']['ReconstructionDataId']))
        astra.algorithm.delete(algorithm_id)
        astra.data3d.delete(cfg['cfg']['alg_cfg']['ReconstructionDataId'])
        astra.data3d.delete(cfg['cfg']['alg_cfg']['ProjectionDataId'])
        astra.functions.clear()
    return np.array(mod_obj)

def fprjr(obj, cfg):
    """
    "forward projector": obj -> prj
    inputs:
        obj: ndarray, 3D array of the object
        projector: str, type of projector in ['tomopy', 'astra']
        *args: positional arg theta for projector 'tomopy', and 'proj_geom', and
               'vol_geom' for projector 'astra'
    """
    if cfg['name'] == 'tomopy':
        proj = tomopy.project(obj, **cfg['cfg'])
    elif cfg['name'] == 'astra':
        fp_id, proj = astra.create_sino3d_gpu(obj, cfg['cfg'])
    return proj

def HIOEngine(inp):
    """
    The convergence is guarantted by the support constraint in obj space and 
    modulus constraint in the projection space. The support constraint in obj
    space uses HIO update scheme.
    Inputs:
        inp: dictionary
            inp['data']: 'prj', 'obj'
            inp['cnts']: 'use_pos_cnt', 'obj_supp'
            inp['prjr']: ['fwd']: 'name', 'cfg' (obj -> prj, 'theta' is defined in 'cfg')
                         ['bac']: 'name', 'cfg' (prj -> obj, 'ang_idx' is define in 'cfg', inp[])
            inp['HIO']: 'beta_obj'
    support constrain is set in slice space; magnitude constrain is set in sinogram space. in the terminology,
    sinogram space = proj space, slice space = obj space

    repeat:
        obj = tomopy.recon(mod_prj)
        prj = tomopy.project(obj)
        mod_prj = Prm(prj)
        mod_obj = tomopy.recon(mod_prj)
        Prm(obj)
            obj = mod_obj                     in support
            obj = obj - beta*mod_obj          not in support
    """
    # forward project (obj - prj), and apply modulus constraint in projection space
    mod_proj = fprjr(inp['data']['obj'], inp['prjr']['fwd'])
    # print(f"{inp['data']['obj'].shape = }, \n{inp['data']['prj'].shape = }, \n{mod_proj.shape = }", '\n',
    #       inp['data']['ang_idx'].shape, inp['prjr']['fwd']['cfg']['theta'].shape)
    mod_proj[inp['data']['ang_idx']] = inp['data']['prj'][inp['data']['ang_idx']]
    mod_proj *= inp['cnts']['sin_supp']

    if inp['cnts']['use_prj_pos_cnt']:
        mod_proj[mod_proj < 0] *= inp['cnts']['prj_pos_f']  
        
    # back project (prj -> obj), and apply support constraint in object space
    mod_obj = bprjr(mod_proj[inp['data']['ang_idx']], inp['prjr']['bac'])
    # print(f"{mod_obj.shape = }, \n{inp['cnts']['obj_supp'].shape = }")
    inp['data']['obj'][:] = (mod_obj * inp['cnts']['obj_supp'] + 
                             (inp['data']['obj'] - inp['HIO']['beta_obj'] * mod_obj) * 
                             (1 - inp['cnts']['obj_supp']))[:]

    
    if inp['cnts']['use_obj_pos_cnt']:
        inp['data']['obj'][inp['data']['obj']<0] *= inp['cnts']['obj_pos_f']
    
    
    """
    ### extra TV iteration -- start
    # forward project (obj - prj), and apply modulus constraint in projection space
    mod_proj = fprjr(inp['data']['obj'], inp['prjr']['fwd'])
    # print(f"{inp['data']['obj'].shape = }, \n{inp['data']['prj'].shape = }, \n{mod_proj.shape = }", '\n',
    #       inp['data']['ang_idx'].shape, inp['prjr']['fwd']['cfg']['theta'].shape)
    mod_proj[inp['data']['ang_idx']] = inp['data']['prj'][inp['data']['ang_idx']]

    if inp['cnts']['use_pos_cnt']:
        mod_proj[mod_proj < 0] *= 0.5  
        
    # back project (prj -> obj), and apply support constraint in object space
    tv_cfg = {'name':'tv', 'cfg':{}}
    tv_cfg['theta'] = cfg['inps']['prjr']['bac']['theta']
    tv_cfg['cfg']['algorithm'] = 'tv'
    tv_cfg['cfg']['center'] = cfg['inps']['prjr']['bac']['cfg']['center']
    tv_cfg['cfg']['num_gridx'] = cfg['inps']['prjr']['bac']['cfg']['num_gridx'] 
    tv_cfg['cfg']['num_gridy'] = cfg['inps']['prjr']['bac']['cfg']['num_gridx'] 
    tv_cfg['cfg']['reg_par'] = 0.3
    tv_cfg['cfg']['init_recon'] = inp['data']['obj']
    
    mod_obj = bprjr(mod_proj, tv_cfg)
    # print(f"{mod_obj.shape = }, \n{inp['cnts']['obj_supp'].shape = }")
    inp['data']['obj'][:] = (mod_obj * inp['cnts']['obj_supp'] + 
                             (inp['data']['obj'] - inp['HIO']['beta_obj'] * mod_obj) * 
                             (1 - inp['cnts']['obj_supp']))[:]
    if inp['cnts']['use_pos_cnt']:
        inp['data']['obj'][inp['data']['obj']<0] *= 0.5
    ### extra TV iteration -- end
    """

    return inp['data']['obj'], mod_proj, mod_obj

def wedge_recon(cfg):
    """
    This is a routine for reconstructing tomography dataset that has a missing
    angle range. The initial inputs of 'proj_modu' and 'theta', however, cover
    the entire 180 deg angle range. The data values in 'proj_modu' in the missing
    angle range are initialized to 0's. These values will be gradually updated 
    in the following iterations.
    
    Inputs:
        cfg: dictionary
            cfg['iter']: dictionary, iteration scheme
                        ['num_updates'], ['update_step'], ['num_HIO_iter']
            cfg['inps']: dictionary, inputs to HIOEngine
                        ['data']: ['prj'], ['obj'], 
                                  ['was'] (wedge angle start), 
                                  ['wae'] (wedge angle end)
                        ['cnts']: ['use_pos_cnt'], ['obj_supp']
                        ['prjr']: ['fwd']: ['name'], ['cfg'] (obj -> prj, 'theta' is defined in 'cfg')
                                  ['bac']: ['name'], ['cfg'] (prj -> obj, inp[])
                        ['HIO']: ['beta_obj']
            cfg['rcfg']: dictionary, debug setting
                        ['rec']: boolean, save intermediate results
                        ['path']: str, where to save intermediate results
    
    Returns:
    """
    if cfg['rcfg']['rec']:
        if not os.path.exists(cfg['rcfg']['path']):
            os.makedirs(cfg['rcfg']['path'], mode=0o777)
            
    cnt = 0
    for ii in range(cfg['iter']['num_updates']):
        p11_s = (cfg['inps']['data']['was'] + cnt*cfg['iter']['update_step'])
        p11_e = (cfg['inps']['data']['was'] + (cnt+1)*cfg['iter']['update_step'])
        # p12_s = (cfg['inps']['data']['was'] + (cnt-1)*cfg['iter']['update_step'])
        # p12_e = (cfg['inps']['data']['was'] + cnt*cfg['iter']['update_step'])
        
        p21_s = (cfg['inps']['data']['wae'] - (cnt+1)*cfg['iter']['update_step'] + 1)
        p21_e = (cfg['inps']['data']['wae'] - cnt*cfg['iter']['update_step'] + 1)
        # p22_s = (cfg['inps']['data']['wae'] - cnt*cfg['iter']['update_step'])
        # p22_e = (cfg['inps']['data']['wae'] - (cnt-1)*cfg['iter']['update_step'])
        print(f'working on angles with indices between {p11_s} and {p11_e}, and {p21_s} and {p21_e}')
                
        for jj in range(cfg['iter']['num_HIO_iter']):
            obj, proj, mod_obj = HIOEngine(cfg['inps'])
            cfg['inps']['prjr']['bac']['cfg']['init_recon'] = mod_obj
            print(f'\t update iteration: {ii}; HIO iteration: {jj}')
            #fn = os.path.join(cfg['rcfg']['path'], f"recon_obj_iter_{str(ii).zfill(3)}_HIO_{str(jj).zfill(3)}.tif")
            #tifffile.imsave(fn, obj.astype(np.float32))
            #fn = os.path.join(cfg['rcfg']['path'], f"recon_prj_iter_{str(ii).zfill(3)}_HIO_{str(jj).zfill(3)}.tif")
            #tifffile.imsave(fn, proj.astype(np.float32))

        """
        for jj in range(cfg['iter']['num_HIO_iter']):
            obj, proj, mod_obj = HIOEngine(cfg['tv_cfg'])
            print(f'\t update iteration: {ii}; TV iteration: {jj}')
        """
        
        print(f"{cfg['inps']['data']['prj'].shape }, \n{proj.shape }")
        cfg['inps']['data']['ang_idx'] = np.concatenate((np.arange(p11_e), np.arange(p21_s+1, cfg['inps']['data']['prj'].shape[0])))
        cfg['inps']['data']['prj'][p11_s:p11_e, :] = proj[p11_s:p11_e, :]
        cfg['inps']['data']['prj'][p21_s:p21_e, :] = proj[p21_s:p21_e, :]
        
        cfg['inps']['prjr']['bac']['theta'] = theta[cfg['inps']['data']['ang_idx']]

        if cfg['rcfg']['rec']:
            fn = os.path.join(cfg['rcfg']['path'], f"recon_obj_iter_{str(ii).zfill(3)}.tif")
            tifffile.imsave(fn, obj.astype(np.float32))
            fn = os.path.join(cfg['rcfg']['path'], f"mod_obj_iter_{str(ii).zfill(3)}.tif")
            tifffile.imsave(fn, mod_obj.astype(np.float32))
            #fn = os.path.join(cfg['rcfg']['path'], f"updated_sino_{str(ii).zfill(3)}.tif")
            #tifffile.imsave(fn, proj)
            fn = os.path.join(cfg['rcfg']['path'], f"updated_prj_iter_{str(ii).zfill(3)}.tif")
            tifffile.imsave(fn, cfg['inps']['data']['prj'].astype(np.float32))
            fn = os.path.join(cfg['rcfg']['path'], f"raw_prj_iter_{str(ii).zfill(3)}.tif")
            tifffile.imsave(fn, proj.astype(np.float32))
            
        if p11_e < p21_s:
            cnt += 1
        else:
            break
    return obj

# ***2. piece tests***

In [ ]:
prj_dim = np.s_[:, 1080:1081, 1005:1595]
obj_dim = [1, 415, 415]
theta_dim = [122, 222]

prj_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66881.h5' 
theta_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66881.h5' 
obj_fn = '' 
obj_supp_fn = '/run/media/xiao/Data/data/Tiankai/66881_1080_mask_415e.h5' 

theta, was, wae = reader(theta_fn, 'theta', dim=theta_dim)   

In [ ]:
prj = reader(prj_fn, 'prj', dim=prj_dim)
obj = reader(obj_fn, 'obj', dim=obj_dim)
supp = reader(obj_supp_fn, 'obj_supp', dim=None)

In [ ]:
prj[was:wae+1, 0, :] = 0
plt.figure(0)
plt.imshow(prj[:, 0, :])

plt.figure(1)
plt.imshow(obj[0])

plt.figure(2)
plt.imshow(supp[:].astype(np.int8))

In [ ]:
plt.figure(0)
plt.imshow(cfg['inps']['cnts']['obj_supp'].astype(np.int8))
#print(cfg['inps']['data']['ang_idx'].shape)
#plt.plot(cfg['inps']['data']['ang_idx'])

In [ ]:
plt.figure(0)
plt.imshow(prj[:, 0, :])
plt.figure(1)
#plt.imshow(prj[:, 0, :]*(fprjr(cfg['inps']['cnts']['obj_supp'].astype(np.int8)[np.newaxis, :], cfg['inps']['prjr']['fwd'])[:, 0, :]>0))
plt.imshow(prj[:, 0, :]*(fprjr(supp[np.newaxis, :], cfg['inps']['prjr']['fwd'])[:, 0, :]>0))
plt.figure(2)
plt.imshow((fprjr(supp[np.newaxis, :], cfg['inps']['prjr']['fwd'])[:, 0, :]>0))

In [ ]:
fn = '/run/media/xiao/Data/data/Tiankai/66881_1080_sinogram_mask_590a.tif' 
tem = tifffile.imread(fn)
prj = np.ndarray([tem.shape[0], 1, tem.shape[1]])
prj[:, 0, :] = tem[:]
fn = '/run/media/xiao/Data/data/Tiankai/66881_1080_prj_mask_590a.tif' 
tifffile.imsave(fn, prj.astype(np.int8))

In [ ]:
cfg['inps']['prjr']['bac']['num_iter'] = 10
cfg['inps']['prjr']['bac']['theta'] = theta
cfg['inps']['prjr']['bac']['name'] = 'tv'
cfg['inps']['prjr']['bac']['cfg']['algorithm'] = 'tv'
cfg['inps']['prjr']['bac']['cfg']['center'] = cen
cfg['inps']['prjr']['bac']['cfg']['num_gridx'] = obj_dim[2] 
cfg['inps']['prjr']['bac']['cfg']['num_gridy'] = obj_dim[1] 
cfg['inps']['prjr']['bac']['cfg']['reg_par'] = 0.1 
obj_supp = bprjr(prj, cfg['inps']['prjr']['bac'])
fn = '/run/media/xiao/Data/data/Tiankai/66881_1080_prj_mask_590a_iradon.tif' 
tifffile.imsave(fn, obj_supp)

In [ ]:
print(prj.shape, theta.shape)
plt.figure(0)
plt.imshow(obj_supp[0])
plt.figure(1)
plt.imshow(prj[:, 0, :])

# ***3. wedge recon***

In [ ]:
"""
prj_dim = np.s_[:, 1080:1081, 930:1520]
obj_dim = [1, 415, 415]
theta_dim = [122, 222]
cen = 303.0
prj_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66885.h5' 
theta_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66885.h5' 
obj_fn = '' 
obj_supp_fn = '/run/media/xiao/Data/data/Tiankai/66885_1080_mask_415e.h5' 
save_path = '/run/media/xiao/Data/data/Tiankai/66885_test_rec'
"""

prj_dim = np.s_[:, 1080:1081, 1005:1595]
obj_dim = [1, 415, 415]
theta_dim = [122, 222]
cen = 307.5
prj_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66881.h5' 
theta_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66881.h5' 
obj_fn = '' 
supp_fn = '/run/media/xiao/Data/data/Tiankai/66881_1080_mask_415e.h5' 
save_path = '/run/media/xiao/Data/data/Tiankai/66881_test_rec_mask_e'

######## initialization mlem round -- start
theta, was, wae = reader(theta_fn, 'theta', dim=theta_dim)   

# config
prj = reader(prj_fn, 'prj', dim=prj_dim)
prj[was:wae+1, 0, :] = 0
ang_idx = np.concatenate((np.arange(was), np.arange(wae+1, prj.shape[0])))

final_obj = tomopy.recon(prj[ang_idx], theta[ang_idx], algorithm='grad', center=cen, 
                       num_gridx=obj_dim[2], num_gridy=obj_dim[1], reg_par=0.1, num_iter=10)
######## initialization mlem round -- end

######## tv round -- start
theta, was, wae = reader(theta_fn, 'theta', dim=theta_dim)   
rec_save_fn = f'{save_path}/tv_round/final_rec.tiff'               

# config
cfg = {'iter':{}, 
       'inps':{'data':{}, 
               'cnts':{}, 
               'prjr':{'fwd':{'cfg':{}}, 
                       'bac':{'cfg':{}}}, 
               'HIO':{}}, 
       'tv_cfg':{'data':{}, 
                 'cnts':{}, 
                 'prjr':{'fwd':{'cfg':{}}, 
                         'bac':{'cfg':{}}}, 
                 'HIO':{}}, 
       'rcfg':{}}

cfg['iter']['num_updates'] = 1 #28
cfg['iter']['update_step'] = 51 #2
cfg['iter']['num_HIO_iter'] = 50

cfg['inps']['data']['prj'] = reader(prj_fn, 'prj', dim=prj_dim)
cfg['inps']['data']['prj'][was:wae+1, 0, :] = 0
cfg['inps']['data']['obj'] = final_obj #reader(obj_fn, 'obj', dim=obj_dim)
cfg['inps']['data']['was'] = was
cfg['inps']['data']['wae'] = wae
cfg['inps']['data']['ang_idx'] = np.concatenate((np.arange(was), np.arange(wae+1, cfg['inps']['data']['prj'].shape[0])))

cfg['inps']['cnts']['use_obj_pos_cnt'] = False
cfg['inps']['cnts']['obj_pos_f'] = 0
cfg['inps']['cnts']['use_prj_pos_cnt'] = False
cfg['inps']['cnts']['prj_pos_f'] = 0.1
cfg['inps']['cnts']['obj_supp'] = reader(supp_fn, 'obj_supp', dim=None)
cfg['inps']['cnts']['sin_supp'] = reader(supp_fn, 'sin_supp', dim=None)[:, np.newaxis, :]

cfg['inps']['prjr']['fwd']['name'] = 'tomopy'
cfg['inps']['prjr']['fwd']['cfg']['theta'] = theta

cfg['inps']['prjr']['bac']['name'] = 'tv'
cfg['inps']['prjr']['bac']['theta'] = theta[cfg['inps']['data']['ang_idx']]
cfg['inps']['prjr']['bac']['cfg']['algorithm'] = 'tv'
cfg['inps']['prjr']['bac']['cfg']['center'] = cen
cfg['inps']['prjr']['bac']['cfg']['num_gridx'] = obj_dim[2] 
cfg['inps']['prjr']['bac']['cfg']['num_gridy'] = obj_dim[1] 
cfg['inps']['prjr']['bac']['cfg']['reg_par'] = 0.1 
#cfg['inps']['prjr']['bac']['cfg']['init_recon'] = final_obj
cfg['inps']['HIO']['beta_obj'] = 0.7

cfg['rcfg']['rec'] = True
cfg['rcfg']['path'] = f'{save_path}/tv_round'

# recon
final_obj = wedge_recon(cfg)

tifffile.imsave(os.path.join(cfg['rcfg']['path'], 'final_rec.tiff'), 
                final_obj.astype(np.float32))
######## tv round -- end


######## mlem round -- start
theta, was, wae = reader(theta_fn, 'theta', dim=theta_dim)   
rec_save_fn = f'{save_path}/mlem/final_rec.tiff'               

# config
cfg = {'iter':{}, 
       'inps':{'data':{}, 
               'cnts':{}, 
               'prjr':{'fwd':{'cfg':{}}, 
                       'bac':{'cfg':{}}}, 
               'HIO':{}}, 
       'tv_cfg':{'data':{}, 
                 'cnts':{}, 
                 'prjr':{'fwd':{'cfg':{}}, 
                         'bac':{'cfg':{}}}, 
                 'HIO':{}}, 
       'rcfg':{}}

cfg['iter']['num_updates'] = 1 #28
cfg['iter']['update_step'] = 51 #2
cfg['iter']['num_HIO_iter'] = 50

cfg['inps']['data']['prj'] = reader(prj_fn, 'prj', dim=prj_dim)
cfg['inps']['data']['prj'][was:wae+1, 0, :] = 0
cfg['inps']['data']['obj'] = final_obj #reader(obj_fn, 'obj', dim=obj_dim)
cfg['inps']['data']['was'] = was
cfg['inps']['data']['wae'] = wae
cfg['inps']['data']['ang_idx'] = np.concatenate((np.arange(was), np.arange(wae+1, cfg['inps']['data']['prj'].shape[0])))

cfg['inps']['cnts']['use_obj_pos_cnt'] = False
cfg['inps']['cnts']['obj_pos_f'] = 0.1
cfg['inps']['cnts']['use_prj_pos_cnt'] = False
cfg['inps']['cnts']['prj_pos_f'] = 0.1
cfg['inps']['cnts']['obj_supp'] = reader(supp_fn, 'obj_supp', dim=None)
cfg['inps']['cnts']['sin_supp'] = reader(supp_fn, 'sin_supp', dim=None)[:, np.newaxis, :]

cfg['inps']['prjr']['fwd']['name'] = 'tomopy'
cfg['inps']['prjr']['fwd']['cfg']['theta'] = theta

cfg['inps']['prjr']['bac']['name'] = 'mlem'
cfg['inps']['prjr']['bac']['theta'] = theta[cfg['inps']['data']['ang_idx']]
cfg['inps']['prjr']['bac']['cfg']['algorithm'] = cfg['inps']['prjr']['bac']['name']
cfg['inps']['prjr']['bac']['cfg']['center'] = cen
cfg['inps']['prjr']['bac']['cfg']['num_gridx'] = obj_dim[2] 
cfg['inps']['prjr']['bac']['cfg']['num_gridy'] = obj_dim[1] 
#cfg['inps']['prjr']['bac']['cfg']['init_recon'] = final_obj

cfg['inps']['HIO']['beta_obj'] = 0.7

cfg['rcfg']['rec'] = True
cfg['rcfg']['path'] = f'{save_path}/mlem'

# recon
final_obj = wedge_recon(cfg)

tifffile.imsave(os.path.join(cfg['rcfg']['path'], 'final_rec.tiff'), 
                final_obj.astype(np.float32))
######## mlem round -- end


# imshow
mpp.imshow(cfg['inps']['data']['obj'][0, :])


mpp.title('rec_wedge_sirt_no_patch')
mpp.show()


In [ ]:
prj_dim = np.s_[:, 1080:1081, 930:1520]
obj_dim = [1, 415, 415]
theta_dim = [122, 222]

prj_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66885.h5' 
theta_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66885.h5' 
obj_fn = '' 
obj_supp_fn = '/run/media/xiao/Data/data/Tiankai/66885_1080_mask_415d.h5' 

theta, was, wae = reader(theta_fn, 'theta', dim=theta_dim)   

rec_save_fn = '/run/media/xiao/Data/data/Tiankai/test_rec/final_rec.tiff'               

# config
cfg = {'iter':{}, 
       'inps':{'data':{}, 
               'cnts':{}, 
               'prjr':{'fwd':{'cfg':{}}, 
                       'bac':{'cfg':{}}}, 
               'HIO':{}}, 
       'tv_cfg':{'data':{}, 
                 'cnts':{}, 
                 'prjr':{'fwd':{'cfg':{}}, 
                         'bac':{'cfg':{}}}, 
                 'HIO':{}}, 
       'rcfg':{}}

cfg['iter']['num_updates'] = 28
cfg['iter']['update_step'] = 2
cfg['iter']['num_HIO_iter'] = 5

cfg['inps']['data']['prj'] = reader(prj_fn, 'prj', dim=prj_dim)
cfg['inps']['data']['prj'][was:wae+1, 0, :] = 0
cfg['inps']['data']['obj'] = reader(obj_fn, 'obj', dim=obj_dim)
cfg['inps']['data']['was'] = was
cfg['inps']['data']['wae'] = wae
cfg['inps']['data']['ang_idx'] = np.concatenate((np.arange(was), np.arange(wae+1, cfg['inps']['data']['prj'].shape[0])))

cfg['inps']['cnts']['use_pos_cnt'] = False
cfg['inps']['cnts']['obj_supp'] = reader(obj_supp_fn, 'obj_supp', dim=None)

cfg['inps']['prjr']['fwd']['name'] = 'tomopy'
cfg['inps']['prjr']['fwd']['cfg']['theta'] = theta

cfg['inps']['prjr']['bac']['name'] = 'mlem'
cfg['inps']['prjr']['bac']['theta'] = theta
cfg['inps']['prjr']['bac']['cfg']['algorithm'] = cfg['inps']['prjr']['bac']['name']
cfg['inps']['prjr']['bac']['cfg']['center'] = 303
cfg['inps']['prjr']['bac']['cfg']['num_gridx'] = obj_dim[2] 
cfg['inps']['prjr']['bac']['cfg']['num_gridy'] = obj_dim[1] 
# cfg['inps']['prjr']['bac']['cfg']['reg_par'] = 0.1 

cfg['inps']['HIO']['beta_obj'] = 0.7

cfg['tv_cfg']['data']['prj'] = reader(prj_fn, 'prj', dim=prj_dim)
cfg['tv_cfg']['data']['prj'][was:wae+1, 0, :] = 0
cfg['tv_cfg']['data']['obj'] = reader(obj_fn, 'obj', dim=obj_dim)
cfg['tv_cfg']['data']['was'] = was
cfg['tv_cfg']['data']['wae'] = wae
cfg['tv_cfg']['data']['ang_idx'] = np.concatenate((np.arange(was), np.arange(wae+1, cfg['inps']['data']['prj'].shape[0])))

cfg['tv_cfg']['cnts']['use_pos_cnt'] = False
cfg['tv_cfg']['cnts']['obj_supp'] = reader(obj_supp_fn, 'obj_supp', dim=None)

cfg['tv_cfg']['prjr']['fwd']['name'] = 'tomopy'
cfg['tv_cfg']['prjr']['fwd']['cfg']['theta'] = theta

cfg['tv_cfg']['prjr']['bac']['name'] = 'tv'
cfg['tv_cfg']['prjr']['bac']['theta'] = theta
cfg['tv_cfg']['prjr']['bac']['cfg']['algorithm'] = 'tv'
cfg['tv_cfg']['prjr']['bac']['cfg']['center'] = 303
cfg['tv_cfg']['prjr']['bac']['cfg']['num_gridx'] = obj_dim[2] 
cfg['tv_cfg']['prjr']['bac']['cfg']['num_gridy'] = obj_dim[1] 
cfg['tv_cfg']['prjr']['bac']['cfg']['reg_par'] = 0.1 
cfg['tv_cfg']['HIO']['beta_obj'] = 0.7

cfg['rcfg']['rec'] = True
cfg['rcfg']['path'] = '/run/media/xiao/Data/data/Tiankai/test_rec'

# recon
cfg['inps']['data']['obj'] = wedge_recon(cfg)

tifffile.imsave(os.path.join(cfg['rcfg']['path'], 'final_rec.tiff'), 
                cfg['inps']['data']['obj'].astype(np.float32))

# imshow
mpp.imshow(cfg['inps']['data']['obj'][0, :])
mpp.title('rec_wedge_sirt_no_patch')
mpp.show()


# ***4. naive tomopy recon***

In [ ]:
prj_dim = np.s_[:, 1080:1081, 930:1520]
obj_dim = [1, 415, 415]
theta_dim = [122, 222]

######## mlem round
prj_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66885.h5' 
theta_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66885.h5' 
obj_fn = '' 
obj_supp_fn = '/run/media/xiao/Data/data/Tiankai/66885_1080_mask_415d.h5' 

theta, was, wae = reader(theta_fn, 'theta', dim=theta_dim)   

rec_save_fn = '/run/media/xiao/Data/data/Tiankai/test_rec/mlem/final_rec.tiff'               

# config
prj = reader(prj_fn, 'prj', dim=prj_dim)
prj[was:wae+1, 0, :] = 0
ang_idx = np.concatenate((np.arange(was), np.arange(wae+1, prj.shape[0])))

mod_obj = tomopy.recon(prj[ang_idx], theta[ang_idx], algorithm='grad', center=303, 
                       num_gridx=obj_dim[2], num_gridy=obj_dim[1], reg_par=0.1, num_iter=50)

fp_cfg = {'name':'tomopy', 'cfg':{'theta':theta}}
mod_prj = fprjr(mod_obj, fp_cfg)

In [ ]:
plt.figure(0)
plt.imshow(mod_obj[0], vmin=0, vmax=7e-3)

plt.figure(1)
plt.imshow(mod_prj[:, 0, :])

plt.figure(2)
plt.imshow(mod_prj[ang_idx, 0, :])

plt.figure(3)
plt.imshow(prj[ang_idx, 0, :])

In [ ]:
plt.plot(theta)

# ***naive astra recon***

In [ ]:
prj_dim = np.s_[:, 1080:1081, 1005:1595]
obj_dim = [1, 415, 415]
ts = 122
theta_dim = [ts, ts+100]
#theta_dim = [122, 222]
#cen = 307.5 - (prj_dim[2].stop - prj_dim[2].start)/2
cen = 290 - (prj_dim[2].stop - prj_dim[2].start)/2
print(f'{cen = }')
prj_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66881.h5' 
theta_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66881.h5' 
obj_fn = '' 
supp_fn = '/run/media/xiao/Data/data/Tiankai/66881_1080_mask_415q.h5' 
rec_save_fn = '/run/media/xiao/Data/data/Tiankai/test_rec/final_rec.tiff' 

theta, was, wae = reader(theta_fn, 'theta', dim=theta_dim)   
#sin_supp = gaussian_filter(reader(supp_fn, 'sin_supp'), 3)
sin_supp = binary_dilation(reader(supp_fn, 'sin_supp').astype(np.float32), disk(1))

#rec_supp = gaussian_filter(binary_dilation(reader(supp_fn, 'obj_supp'), disk(3)).astype(np.float32), 3)
rec_supp = binary_dilation(reader(supp_fn, 'obj_supp'), disk(20))
#rec_supp = gaussian_filter(reader(supp_fn, 'obj_supp').astype(np.float32), 5)
#rec_supp = reader(supp_fn, 'obj_supp')


prj = np.squeeze(reader(prj_fn, 'prj', dim=prj_dim))
prj -= ((prj.sum(axis=1)-220)/prj.shape[1])[:, np.newaxis]
#prj[:] = gaussian_filter(prj, 3)[:]
prj[was:wae+1, :] = 0
ang_idx = np.concatenate((np.arange(was), np.arange(wae+1, prj.shape[0])))

#prj = tifffile.imread('/media/xiao_usb2/Tiankai/66881_1080_mask_415q_sinogram_bin.tif')
print(f"{prj.shape = }, {prj[ang_idx].shape = }, {theta[ang_idx].shape = }")

In [ ]:
print(f"    recon starts at {time.asctime()}")
prj_geom_s = astra.create_proj_geom('parallel', 1., 590, theta[ang_idx])
prj_geom_s_cor = astra.functions.geom_postalignment(prj_geom_s, cen)
vol_geom = astra.create_vol_geom(415, 415)

rec_id = astra.data2d.create('-vol', vol_geom, 0)
rec_supp_id = astra.data2d.create('-vol', vol_geom, rec_supp)
sin_s_id = astra.data2d.create('-sino', prj_geom_s_cor, prj[ang_idx])
sin_s_supp_id = astra.data2d.create('-sino', prj_geom_s_cor, sin_supp[ang_idx])

#proj_geom_vec = astra.functions.geom_2vec(proj_geom_cor)
#proj_geom_vec['Vectors'][:, 11] *= np.sin(np.pi/2-alpha)
#proj_id = astra.data3d.create('-sino', proj_geom_vec, imgs)

alg_s_cfg = astra.astra_dict('SIRT_CUDA')
alg_s_cfg['ProjectionDataId'] = sin_s_id 
alg_s_cfg['ReconstructionDataId'] = rec_id
alg_s_cfg['option'] = {}
alg_s_cfg['option']['SinogramMaskId'] = sin_s_supp_id
alg_s_cfg['option']['ReconstructionMaskId'] = rec_supp_id
alg_s_cfg['option']['MaxConstraint'] = 2e-2
alg_s_cfg['option']['MinConstraint'] = -5e-4

alg_s_id = astra.algorithm.create(alg_s_cfg)

prj_geom_f = astra.create_proj_geom('parallel', 1., 590, theta)
prj_geom_f_cor = astra.functions.geom_postalignment(prj_geom_f, cen)
sin_f_id = astra.data2d.create('-sino', prj_geom_f_cor, prj)
sin_f_supp_id = astra.data2d.create('-sino', prj_geom_f_cor, sin_supp)

alg_f_cfg = astra.astra_dict('CGLS_CUDA')
alg_f_cfg['ProjectionDataId'] = sin_f_id 
alg_f_cfg['ReconstructionDataId'] = rec_id
alg_f_cfg['option'] = {}
#alg_f_cfg['option']['SinogramMaskId'] = sin_f_supp_id
alg_f_cfg['option']['ReconstructionMaskId'] = rec_supp_id
alg_f_cfg['option']['MaxConstraint'] = 2e-2
alg_f_cfg['option']['MinConstraint'] = -5e-4

alg_f_id = astra.algorithm.create(alg_f_cfg)

#tv = np.zeros([415, 415], dtype=np.float32)
ts = np.zeros(prj.shape, dtype=np.float32)
proj_id = astra.create_projector('cuda', prj_geom_f_cor, vol_geom)
W = astra.OpTomo(proj_id)
for ii in range(100):
    ss = astra.data2d.get_shared(sin_s_id)    
    median_filter(ss, [1, 3], output=ss)
    astra.algorithm.run(alg_s_id, 10)
    
    vs = astra.data2d.get_shared(rec_id)
    sf = astra.data2d.get_shared(sin_f_id) 
    ts[:] = W.FP(vs)[:]
    sf[theta_dim[0]:theta_dim[1]+1] = ts[theta_dim[0]:theta_dim[1]+1]
    astra.algorithm.run(alg_f_id, 3)           
    median_filter(sf, [1, 3], output=sf)
    #gaussian_filter(sf[theta_dim[0]:theta_dim[1]+1], 3, output=sf[theta_dim[0]:theta_dim[1]+1])
    astra.algorithm.run(alg_f_id, 3)  

recon = astra.data2d.get(rec_id)
print(f"    recon finishes at {time.asctime()}")

#astra_mex_data2d('change_geometry', id, geom)
fp_geom = astra.create_proj_geom('parallel', 1., 590, theta)
fp_id = astra.create_projector('cuda', fp_geom, vol_geom)
sinogram_id, sinogram = astra.create_sino(recon, fp_id)
print(f"    forward projection finishes at {time.asctime()}")

"""
ss = astra.data2d.get_shared(sin_s_id)    
ss[:] = median_filter(ss, [1, 5])
tifffile.imsave('/media/xiao_usb2/Tiankai/66881_1080_391x590_sino_s.tif', ss.astype(np.float32))

sf = astra.data2d.get_shared(sin_f_id)    
sf[:] = median_filter(sf, [1, 5])
tifffile.imsave('/media/xiao_usb2/Tiankai/66881_1080_391x590_sino_f.tif', sf.astype(np.float32))
"""

astra.algorithm.delete(alg_s_id)
astra.data2d.delete(sin_s_id)
astra.data2d.delete(sin_s_supp_id)

astra.algorithm.delete(alg_f_id)
astra.data2d.delete(sin_f_id)
astra.data2d.delete(sin_f_supp_id)

astra.data2d.delete(rec_id)
astra.data2d.delete(rec_supp_id)

astra.data2d.delete(sinogram_id)
astra.projector.delete(fp_id)
astra.functions.clear()

In [ ]:
print(f"    recon starts at {time.asctime()}")
proj_geom = astra.create_proj_geom('parallel', 1., 590, theta[ang_idx])
#proj_geom = astra.create_proj_geom('parallel', 1., 590, theta)
proj_geom_cor = astra.functions.geom_postalignment(proj_geom, cen)
vol_geom = astra.create_vol_geom(415, 415)

recon_id = astra.data2d.create('-vol', vol_geom, 0)
rec_supp_id = astra.data2d.create('-vol', vol_geom, rec_supp)
sino_id = astra.data2d.create('-sino', proj_geom_cor, prj[ang_idx])
sin_supp_id = astra.data2d.create('-sino', proj_geom_cor, sin_supp[ang_idx])
#sino_id = astra.data2d.create('-sino', proj_geom_cor, prj)
#sin_supp_id = astra.data2d.create('-sino', proj_geom_cor, sin_supp)

#proj_geom_vec = astra.functions.geom_2vec(proj_geom_cor)
#proj_geom_vec['Vectors'][:, 11] *= np.sin(np.pi/2-alpha)
#proj_id = astra.data3d.create('-sino', proj_geom_vec, imgs)

alg_cfg = astra.astra_dict('SIRT_CUDA')
#alg_cfg = astra.astra_dict('SIRT3D_CUDA')
#alg_cfg = astra.astra_dict('BP3D_CUDA')

alg_cfg['ProjectionDataId'] = sino_id #sinogram_id #
alg_cfg['ReconstructionDataId'] = recon_id
alg_cfg['option'] = {}
alg_cfg['option']['SinogramMaskId'] = sin_supp_id
alg_cfg['option']['ReconstructionMaskId'] = rec_supp_id
alg_cfg['option']['MaxConstraint'] = 2e-2
#alg_cfg['option']['MinConstraint'] = -5e-4

algorithm_id = astra.algorithm.create(alg_cfg)
astra.algorithm.run(algorithm_id, 1000)

#for ii in range(100):
#    astra.algorithm.run(algorithm_id, 10)


recon = astra.data2d.get(recon_id)
print(f"    recon finishes at {time.asctime()}")

#astra_mex_data2d('change_geometry', id, geom)
fp_geom = astra.create_proj_geom('parallel', 1., 590, theta)
fp_id = astra.create_projector('cuda', fp_geom, vol_geom)
sinogram_id, sinogram = astra.create_sino(recon, fp_id)
print(f"    forward projection finishes at {time.asctime()}")

astra.algorithm.delete(algorithm_id)
astra.data2d.delete(recon_id)
astra.data2d.delete(sino_id)
astra.data2d.delete(sinogram_id)
astra.data2d.delete(proj_id)
astra.projector.delete(fp_id)
astra.functions.clear()

In [ ]:
plt.figure(10)
plt.imshow(recon, vmin=-1e-4, vmax=10e-3)
#plt.imshow(rec_supp)

plt.figure(11)
plt.imshow(sinogram, vmin=0, vmax=3)

plt.figure(12)
plt.imshow(prj, vmin=0, vmax=3)

plt.figure(13)
plt.imshow(prj*sinogram, vmin=0, vmax=3)
#plt.imshow(prj*(sinogram!=0))
#plt.imshow(rec_supp)

plt.figure(14)
plt.imshow(prj-sinogram, vmin=0, vmax=3)

In [ ]:
plt.figure(0)
plt.imshow(recon, vmin=-1e-4, vmax=10e-3)
#plt.imshow(rec_supp)

plt.figure(1)
plt.imshow(sinogram, vmin=0, vmax=2)

plt.figure(2)
plt.imshow(prj, vmin=0, vmax=3)

plt.figure(3)
plt.imshow(prj*(sinogram!=0))
#plt.imshow(rec_supp)

plt.figure(4)
plt.imshow(prj+sinogram, vmin=0, vmax=3)

In [ ]:
plt.figure(5)
plt.plot(sinogram.sum(axis=1))
#plt.plot(sinogram.sum(axis=1))

In [ ]:
prj = tifffile.imread('/media/xiao_usb2/Tiankai/66881_1080_sinogram_590a_mask.tif')

In [ ]:
#tifffile.imsave('/media/xiao_usb2/Tiankai/66881_1080_mask_415q_sinogram.tif', (sinogram+prj).astype(np.float32))
tifffile.imsave('/media/xiao_usb2/Tiankai/66881_1080_mask_415q_sinogram_bin_recon.tif', recon.astype(np.float32))
#tifffile.imsave('/run/media/xiao/Data/data/Tiankai/66881_1080_sinogram_mask_590f.tif', (prj-sinogram).astype(np.float32))

In [ ]:
import scipy.io
P = scipy.io.loadmat('/home/xiao/Downloads/phantom.mat')['phantom256']
plt.imshow(P)


In [ ]:
help(gaussian_filter)
#dir(astra)
#help(astra.create_sino)

In [ ]:
print(rec_supp)
print(reader(supp_fn, 'obj_supp'))

In [ ]:
plt.figure(20)
plt.imshow(sin_supp)

In [ ]:
dir(vol_geom)

In [ ]:
help(vol_geom.items)

# ***naive astra with phantom***

In [ ]:
prj_dim = np.s_[:, 1080:1081, 1005:1595]
obj_dim = [1, 415, 415]
ts = 122
theta_dim = [ts, ts+100]
#theta_dim = [122, 222]
#cen = 307.5 - (prj_dim[2].stop - prj_dim[2].start)/2
cen = 290 - (prj_dim[2].stop - prj_dim[2].start)/2
print(f'{cen = }')
prj_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66881.h5' 
theta_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66881.h5' 
obj_fn = '' 
supp_fn = '/run/media/xiao/Data/data/Tiankai/66881_1080_mask_415q.h5' 
rec_save_fn = '/run/media/xiao/Data/data/Tiankai/test_rec/final_rec.tiff' 

theta, was, wae = reader(theta_fn, 'theta', dim=theta_dim)   
#sin_supp = gaussian_filter(reader(supp_fn, 'sin_supp'), 3)
sin_supp = binary_dilation(reader(supp_fn, 'sin_supp').astype(np.float32), disk(1))

#rec_supp = gaussian_filter(binary_dilation(reader(supp_fn, 'obj_supp'), disk(3)).astype(np.float32), 3)
rec_supp = binary_dilation(reader(supp_fn, 'obj_supp'), disk(20))
#rec_supp = gaussian_filter(reader(supp_fn, 'obj_supp').astype(np.float32), 5)
#rec_supp = reader(supp_fn, 'obj_supp')


prj = np.squeeze(reader(prj_fn, 'prj', dim=prj_dim))
prj -= ((prj.sum(axis=1)-220)/prj.shape[1])[:, np.newaxis]
#prj[:] = gaussian_filter(prj, 3)[:]
prj[was:wae+1, :] = 0
ang_idx = np.concatenate((np.arange(was), np.arange(wae+1, prj.shape[0])))


#prj = tifffile.imread('/media/xiao_usb2/Tiankai/66881_1080_mask_415q_sinogram_bin.tif')
print(f"{prj.shape = }, {prj[ang_idx].shape = }, {theta[ang_idx].shape = }")

In [ ]:
theta_dim = [60, 120]
supp_rel = 10
pha =io.loadmat('/run/media/xiao/Data/data/Tiankai/phantom.mat')['phantom256']
rec_supp = binary_dilation((pha!=0).astype(np.int8), disk(supp_rel))
rec_supp = (recon>0)
#rec_supp = binary_erosion(recon>0, disk(1))

theta = np.linspace(0,np.pi,180,False)
ang_idx = np.concatenate((np.arange(theta_dim[0]), np.arange(theta_dim[1]+1, 180)))

vol_geom = astra.create_vol_geom(256, 256)
prj_geom_f_cor = astra.create_proj_geom('parallel', 1.0, 384, theta)
prj_geom_s_cor = astra.create_proj_geom('parallel', 1.0, 384, theta[ang_idx])

prj_id = astra.create_projector('cuda', prj_geom_f_cor, vol_geom)
sin_f_id, prj = astra.create_sino(pha, prj_id)
sin_supp = binary_dilation(prj, disk(supp_rel))
sin_supp = (sinogram>0)
sin_f_supp_id = astra.data2d.create('-sino', prj_geom_f_cor, sin_supp)
astra.data2d.get_shared(sin_f_id)
sf = astra.data2d.get_shared(sin_f_id) 
sf[theta_dim[0]:theta_dim[1]+1] = 0
#sin_f_id = astra.data2d.create('-sino', prj_geom_f_cor, prj)

rec_id = astra.data2d.create('-vol', vol_geom, 0)
rec_supp_id = astra.data2d.create('-vol', vol_geom, rec_supp)
sin_s_id = astra.data2d.create('-sino', prj_geom_s_cor, prj[ang_idx])
sin_s_supp_id = astra.data2d.create('-sino', prj_geom_s_cor, sin_supp[ang_idx])

alg_s_cfg = astra.astra_dict('SIRT_CUDA')
alg_s_cfg['ProjectionDataId'] = sin_s_id 
alg_s_cfg['ReconstructionDataId'] = rec_id
alg_s_cfg['option'] = {}
alg_s_cfg['option']['SinogramMaskId'] = sin_s_supp_id
alg_s_cfg['option']['ReconstructionMaskId'] = rec_supp_id
#alg_s_cfg['option']['MaxConstraint'] = 2e-2
#alg_s_cfg['option']['MinConstraint'] = -5e-4

alg_s_id = astra.algorithm.create(alg_s_cfg)



alg_f_cfg = astra.astra_dict('SART_CUDA')
alg_f_cfg['ProjectionDataId'] = sin_f_id 
alg_f_cfg['ReconstructionDataId'] = rec_id
alg_f_cfg['option'] = {}
#alg_f_cfg['option']['SinogramMaskId'] = sin_f_supp_id
alg_f_cfg['option']['ReconstructionMaskId'] = rec_supp_id
#alg_f_cfg['option']['MaxConstraint'] = 2e-2
#alg_f_cfg['option']['MinConstraint'] = -5e-4

alg_f_id = astra.algorithm.create(alg_f_cfg)

#tv = np.zeros([415, 415], dtype=np.float32)
ts = np.zeros(prj.shape, dtype=np.float32)
W = astra.OpTomo(prj_id)
for ii in range(100):
    ss = astra.data2d.get_shared(sin_s_id)    
    median_filter(ss, [1, 3], output=ss)
    astra.algorithm.run(alg_s_id, 10)
    
    vs = astra.data2d.get_shared(rec_id)
    sf = astra.data2d.get_shared(sin_f_id) 
    ts[:] = W.FP(vs)[:]
    sf[theta_dim[0]:theta_dim[1]+1] = ts[theta_dim[0]:theta_dim[1]+1]
    astra.algorithm.run(alg_f_id, 3)           
    median_filter(sf, [1, 3], output=sf)
    #gaussian_filter(sf[theta_dim[0]:theta_dim[1]+1], 3, output=sf[theta_dim[0]:theta_dim[1]+1])
    astra.algorithm.run(alg_f_id, 3)  

recon = astra.data2d.get(rec_id)
print(f"    recon finishes at {time.asctime()}")

#astra_mex_data2d('change_geometry', id, geom)
fp_geom = astra.create_proj_geom('parallel', 1., 384, theta)
fp_id = astra.create_projector('cuda', fp_geom, vol_geom)
sinogram_id, sinogram = astra.create_sino(recon, fp_id)
print(f"    forward projection finishes at {time.asctime()}")

"""
ss = astra.data2d.get_shared(sin_s_id)    
ss[:] = median_filter(ss, [1, 5])
tifffile.imsave('/media/xiao_usb2/Tiankai/66881_1080_391x590_sino_s.tif', ss.astype(np.float32))

sf = astra.data2d.get_shared(sin_f_id)    
sf[:] = median_filter(sf, [1, 5])
tifffile.imsave('/media/xiao_usb2/Tiankai/66881_1080_391x590_sino_f.tif', sf.astype(np.float32))
"""

astra.algorithm.delete(alg_s_id)
astra.data2d.delete(sin_s_id)
astra.data2d.delete(sin_s_supp_id)

astra.algorithm.delete(alg_f_id)
astra.data2d.delete(sin_f_id)
astra.data2d.delete(sin_f_supp_id)

astra.data2d.delete(rec_id)
astra.data2d.delete(rec_supp_id)

astra.data2d.delete(sinogram_id)
astra.projector.delete(fp_id)
astra.projector.delete(prj_id)
astra.functions.clear()

In [ ]:
plt.imshow(sin_supp)

In [ ]:
plt.figure(0)
plt.imshow(recon)
#plt.imshow(recon, vmin=-1e-4, vmax=10e-3)
#plt.imshow(rec_supp)

plt.figure(1)
plt.imshow(sinogram)
#plt.imshow(sinogram, vmin=0, vmax=2)

plt.figure(2)
plt.imshow(prj)
#plt.imshow(prj, vmin=0, vmax=3)

plt.figure(3)
plt.imshow(prj*(sinogram!=0))
#plt.imshow(rec_supp)

plt.figure(4)
plt.imshow(prj+sinogram)
#plt.imshow(prj+sinogram, vmin=0, vmax=3)

In [ ]:
plt.figure(5)
plt.imshow(recon*(recon>0))

plt.figure(6)
plt.imshow(prj*(sinogram>0))

# ***naive astra recon on real data -- clean version***

In [26]:
sin_thres = 0.15
sin_supp_rel = 10
rec_thres = 2.5e-4
rec_supp_rel = 15

#prj_dim = np.s_[:, 250:251, 123:677]
prj_dim = np.s_[:, 270:271, 123:580]
obj_dim = [415, 415]
recon = np.zeros(obj_dim)

#cen = 48.5+277 - (prj_dim[2].stop - prj_dim[2].start)/2
cen = 0
print(f'{cen = }')
prj_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_6797.h5' 
theta_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_6797.h5' 
obj_fn = '' 
with h5py.File(theta_fn, 'r') as f:
    mar = f['missing_ang_range'][:]
    
rec_supp_fn = '/run/media/xiao/Data/data/Tiankai/6797/6797_roi_250_mask_415b_recon_mask.tif'
rec_supp = tifffile.imread(rec_supp_fn)
rec_supp = binary_dilation(rec_supp, disk(rec_supp_rel))
sin_supp_fn = '/run/media/xiao/Data/data/Tiankai/6797/6797_roi_250_mask_415c_sino_mask.tif'
sin_supp = tifffile.imread(sin_supp_fn)
sin_supp = binary_dilation(sin_supp, disk(sin_supp_rel))

#supp_fn = '/run/media/xiao/Data/data/Tiankai/66881_1080_mask_415q.h5' 
#rec_save_fn = '/run/media/xiao/Data/data/Tiankai/test_rec/final_rec.tiff' 

theta, was, wae = reader(theta_fn, 'theta', dim=mar)   
#sin_supp = gaussian_filter(reader(supp_fn, 'sin_supp'), 3)
#sin_supp = binary_dilation(reader(supp_fn, 'sin_supp').astype(np.float32), disk(1))

#rec_supp = gaussian_filter(binary_dilation(reader(supp_fn, 'obj_supp'), disk(3)).astype(np.float32), 3)
#rec_supp = binary_dilation(reader(supp_fn, 'obj_supp'), disk(20))
#rec_supp = gaussian_filter(reader(supp_fn, 'obj_supp').astype(np.float32), 5)
#rec_supp = reader(supp_fn, 'obj_supp')


prj = np.squeeze(reader(prj_fn, 'prj', dim=prj_dim))
-np.log(prj, out=prj)
prj[prj<0] = 0
prj[np.isinf(prj)] = 0
prj[np.isnan(prj)] = 0
#plt.figure(100)
#plt.plot(prj.sum(axis=1))
prj -= ((prj.sum(axis=1)-90)/prj.shape[1])[:, np.newaxis]
prj[was:wae+1, :] = 0
prj[prj<0] = 0
median_filter(prj, size=[1, 5], output=prj)
#plt.figure(101)
#plt.plot(prj.sum(axis=1))
ang_idx = np.concatenate((np.arange(was), np.arange(wae+1, prj.shape[0])))

sin_supp[ang_idx] = (prj[ang_idx]>0).astype(np.int8)

#prj = tifffile.imread('/media/xiao_usb2/Tiankai/66881_1080_mask_415q_sinogram_bin.tif')
print(f"{prj.shape = }, {prj[ang_idx].shape = }, {theta[ang_idx].shape = }")



cen = 0
prj.shape = (386, 457), prj[ang_idx].shape = (281, 457), theta[ang_idx].shape = (281,)


<ipython-input-2-4b71a91cd265>:16: RuntimeWarning: divide by zero encountered in log
  tem = -np.log((f['/img_tomo'][dim] - f['/img_dark_avg'][dim])/(f['/img_bkg_avg'][dim] - f['/img_dark_avg'][dim]))
<ipython-input-26-70ecbe68d1c6>:41: RuntimeWarning: invalid value encountered in log
  -np.log(prj, out=prj)


In [27]:
vol_geom = astra.create_vol_geom(*obj_dim)
prj_geom_f = astra.create_proj_geom('parallel', 1.0, prj.shape[1], theta)
prj_geom_f_cor = astra.functions.geom_postalignment(prj_geom_f, cen)
prj_geom_s = astra.create_proj_geom('parallel', 1.0, prj.shape[1], theta[ang_idx])
prj_geom_s_cor = astra.functions.geom_postalignment(prj_geom_s, cen)

prj_id = astra.create_projector('cuda', prj_geom_f_cor, vol_geom)
#sin_f_id, prj = astra.create_sino(pha, prj_id)
sin_f_id = astra.data2d.create('-sino', prj_geom_f_cor, prj)
sin_f_supp_id = astra.data2d.create('-sino', prj_geom_f_cor, sin_supp)
sf = astra.data2d.get_shared(sin_f_id) 
sf[mar[0]:mar[1]+1] = 0


rec_id = astra.data2d.create('-vol', vol_geom, 0)
rec_supp_id = astra.data2d.create('-vol', vol_geom, rec_supp)
sin_s_id = astra.data2d.create('-sino', prj_geom_s_cor, prj[ang_idx])
sin_s_supp_id = astra.data2d.create('-sino', prj_geom_s_cor, sin_supp[ang_idx])

if True:
    alg_s_cfg = astra.astra_dict('SIRT_CUDA')
    alg_s_cfg['ProjectionDataId'] = sin_s_id 
    alg_s_cfg['ReconstructionDataId'] = rec_id
    alg_s_cfg['option'] = {}
    #alg_s_cfg['option']['SinogramMaskId'] = sin_s_supp_id
    alg_s_cfg['option']['ReconstructionMaskId'] = rec_supp_id
    alg_s_cfg['option']['MaxConstraint'] = 2e-2
    alg_s_cfg['option']['MinConstraint'] = -1e-4
    alg_s_id = astra.algorithm.create(alg_s_cfg)

    alg_f_cfg = astra.astra_dict('SIRT_CUDA')
    alg_f_cfg['ProjectionDataId'] = sin_f_id 
    alg_f_cfg['ReconstructionDataId'] = rec_id
    alg_f_cfg['option'] = {}
    #alg_f_cfg['option']['SinogramMaskId'] = sin_f_supp_id
    alg_f_cfg['option']['ReconstructionMaskId'] = rec_supp_id
    alg_f_cfg['option']['MaxConstraint'] = 2e-2
    alg_f_cfg['option']['MinConstraint'] = -1e-4
    alg_f_id = astra.algorithm.create(alg_f_cfg)

    #tv = np.zeros([415, 415], dtype=np.float32)
    ts = np.zeros(prj.shape, dtype=np.float32)
    W = astra.OpTomo(prj_id)
    for ii in range(100):
        ss = astra.data2d.get_shared(sin_s_id)    
        median_filter(ss, [1, 3], output=ss)
        astra.algorithm.run(alg_s_id, 10)

        vs = astra.data2d.get_shared(rec_id)
        sf = astra.data2d.get_shared(sin_f_id) 
        ts[:] = W.FP(vs)[:]
        sf[mar[0]:mar[1]+1] = ts[mar[0]:mar[1]+1]
        astra.algorithm.run(alg_f_id, 3)           
        median_filter(sf, [1, 3], output=sf)
        #gaussian_filter(sf[theta_dim[0]:theta_dim[1]+1], 3, output=sf[theta_dim[0]:theta_dim[1]+1])
        astra.algorithm.run(alg_f_id, 3)  
        """
        recon[:] = astra.data2d.get(rec_id)[:]
        sinogram_id, sino = astra.create_sino(recon, prj_id)
        ssf = astra.data2d.get_shared(sin_f_supp_id) 
        ssf[:] = (sino>0.25).astype(np.int8)[:]
        ssf[ang_idx] = (prj[ang_idx]>0).astype(np.int8)[:]
        sss = astra.data2d.get_shared(sin_s_supp_id) 
        sss[:] = ssf[[ang_idx]]
        astra.data2d.delete(sinogram_id)
        """

    recon[:] = astra.data2d.get(rec_id)[:]
    rs = astra.data2d.get_shared(rec_supp_id) 
    rs[:] = (gaussian_filter(recon, rec_supp_rel)>rec_thres).astype(np.int8)[:]
    if True:
        sinogram_id, sino = astra.create_sino(recon, prj_id)
        ssf = astra.data2d.get_shared(sin_f_supp_id) 
        ssf[:] = (sino>sin_thres).astype(np.int8)[:]
        ssf[ang_idx] = (prj[ang_idx]>0).astype(np.int8)[:]
        ssf[:] = binary_dilation(ssf, disk(sin_supp_rel))[:]
        sss = astra.data2d.get_shared(sin_s_supp_id) 
        sss[:] = ssf[[ang_idx]]
        astra.data2d.delete(sinogram_id)

if True:
    astra.data2d.delete(alg_s_id)
    astra.data2d.delete(alg_f_id)

    alg_s_cfg = astra.astra_dict('SIRT_CUDA')
    alg_s_cfg['ProjectionDataId'] = sin_s_id 
    alg_s_cfg['ReconstructionDataId'] = rec_id
    alg_s_cfg['option'] = {}
    #alg_s_cfg['option']['SinogramMaskId'] = sin_s_supp_id
    alg_s_cfg['option']['ReconstructionMaskId'] = rec_supp_id
    alg_s_cfg['option']['MaxConstraint'] = 2e-2
    #alg_s_cfg['option']['MinConstraint'] = -1e-4
    alg_s_id = astra.algorithm.create(alg_s_cfg)

    alg_f_cfg = astra.astra_dict('SIRT_CUDA')
    alg_f_cfg['ProjectionDataId'] = sin_f_id 
    alg_f_cfg['ReconstructionDataId'] = rec_id
    alg_f_cfg['option'] = {}
    #alg_f_cfg['option']['SinogramMaskId'] = sin_f_supp_id
    alg_f_cfg['option']['ReconstructionMaskId'] = rec_supp_id
    #alg_f_cfg['option']['MaxConstraint'] = 2e-2
    alg_f_cfg['option']['MinConstraint'] = -1e-4
    alg_f_id = astra.algorithm.create(alg_f_cfg)

    for ii in range(100):
        ss = astra.data2d.get_shared(sin_s_id)    
        median_filter(ss, [1, 3], output=ss)
        astra.algorithm.run(alg_s_id, 10)

        vs = astra.data2d.get_shared(rec_id)
        sf = astra.data2d.get_shared(sin_f_id) 
        ts[:] = W.FP(vs)[:]
        sf[mar[0]:mar[1]+1] = ts[mar[0]:mar[1]+1]
        astra.algorithm.run(alg_f_id, 3)           
        median_filter(sf, [1, 3], output=sf)
        #gaussian_filter(sf[theta_dim[0]:theta_dim[1]+1], 3, output=sf[theta_dim[0]:theta_dim[1]+1])
        astra.algorithm.run(alg_f_id, 3)  


recon[:] = astra.data2d.get(rec_id)[:]
print(f"    recon finishes at {time.asctime()}")

#astra_mex_data2d('change_geometry', id, geom)
#fp_geom = astra.create_proj_geom('parallel', 1., prj.shape[1], theta)
#fp_id = astra.create_projector('cuda', fp_geom, vol_geom)
sinogram_id, sino = astra.create_sino(recon, prj_id)
print(f"    forward projection finishes at {time.asctime()}")

"""
ss = astra.data2d.get_shared(sin_s_id)    
ss[:] = median_filter(ss, [1, 5])
tifffile.imsave('/media/xiao_usb2/Tiankai/66881_1080_391x590_sino_s.tif', ss.astype(np.float32))

sf = astra.data2d.get_shared(sin_f_id)    
sf[:] = median_filter(sf, [1, 5])
tifffile.imsave('/media/xiao_usb2/Tiankai/66881_1080_391x590_sino_f.tif', sf.astype(np.float32))
"""
if False:
    plt.figure(100)
    plt.imshow(ssf)
    plt.figure(101)
    plt.imshow(sss)


astra.algorithm.delete(alg_s_id)
astra.data2d.delete(sin_s_id)
astra.data2d.delete(sin_s_supp_id)

astra.algorithm.delete(alg_f_id)
astra.data2d.delete(sin_f_id)
astra.data2d.delete(sin_f_supp_id)

astra.data2d.delete(rec_id)
astra.data2d.delete(rec_supp_id)

astra.data2d.delete(sinogram_id)
#astra.projector.delete(fp_id)
astra.projector.delete(prj_id)
astra.functions.clear()


<ipython-input-27-1108466a1348>:78: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sss[:] = ssf[[ang_idx]]


    recon finishes at Sat Dec 12 15:40:29 2020
    forward projection finishes at Sat Dec 12 15:40:29 2020


In [ ]:
#plt.figure(10)
#plt.imshow(astra.data2d.get_shared(rec_supp_id))
plt.figure(11)
#prj = np.squeeze(reader(prj_fn, 'prj', dim=prj_dim))
#plt.plot(prj.sum(axis=1))
#plt.plot(sinogram.sum(axis=1))
plt.imshow(ssf)
#plt.imshow(binary_dilation(ssf, disk(5)))

In [31]:
plt.figure(0)
#plt.imshow(recon)
plt.imshow(recon, vmin=-10e-3, vmax=1e-2)
plt.gca().set_title('recon')
#plt.imshow(rec_supp)

plt.figure(1)
plt.imshow(sino)
plt.gca().set_title('sinogram')
#plt.imshow(sinogram, vmin=0, vmax=2)

plt.figure(2)
plt.imshow(prj)
plt.gca().set_title('prj')
#plt.imshow(prj, vmin=0, vmax=3)

plt.figure(3)
plt.imshow(rec_supp)
plt.gca().set_title('prj>0')
#plt.imshow(rec_supp)

plt.figure(4)
plt.imshow(prj+sino)
plt.gca().set_title('prj+sinogram')
#plt.imshow(prj+sinogram, vmin=0,

plt.figure(5)
#plt.imshow(sino>1e-1)
plt.imshow(binary_dilation((sino>sin_thres).astype(np.int8), disk(sin_supp_rel)))
#plt.imshow((sino>0.25e-1).astype(np.int8))
plt.gca().set_title('prj>sin_thres')

#plt.figure(6)
#plt.imshow((prj))

Text(0.5, 1.0, 'prj>sin_thres')

In [ ]:
tem = gaussian_filter(recon, 15)
plt.figure(200)
plt.imshow(tem>1e-3)
plt.figure(201)
#plt.imshow(np.abs(np.diff(tem, axis=0, prepend=1))+np.abs(np.diff(tem, axis=1, prepend=1)),
#           vmin=-1e-4, vmax=1e-4)
plt.imshow((tem>1e-3)*recon)
#plt.hist(tem, 10)

In [ ]:
tifffile.imsave('/run/media/xiao/Data/data/Tiankai/6797/6797_roi_250_mask_415c_recon.tif', recon.astype(np.float32))

In [ ]:
tifffile.imsave('/run/media/xiao/Data/data/Tiankai/6797/6797_roi_250_mask_415b_sino.tif', sinogram.astype(np.float32))

# ***astra volume recon***

In [ ]:
### prepare data
sin_thres = 0.15
sin_supp_rel = 10
rec_thres = 2.5e-4
rec_supp_rel = 15

#prj_dim = np.s_[:, 250:251, 123:677]
prj_dim = np.s_[:, 200:300, 123:580]
obj_dim = [415, 415]

cen = 0
print(f'{cen = }')
prj_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_6797.h5' 
theta_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_6797.h5' 
obj_fn = '' 
with h5py.File(theta_fn, 'r') as f:
    mar = f['missing_ang_range'][:]
    
rec_supp_fn = '/run/media/xiao/Data/data/Tiankai/6797/6797_roi_250_mask_415b_recon_mask.tif'
rec_supp = tifffile.imread(rec_supp_fn)
rec_supp = binary_dilation(rec_supp, disk(rec_supp_rel))
sin_supp_fn = '/run/media/xiao/Data/data/Tiankai/6797/6797_roi_250_mask_415c_sino_mask.tif'
sin_supp = tifffile.imread(sin_supp_fn)
sin_supp = binary_dilation(sin_supp, disk(sin_supp_rel))

theta, was, wae = reader(theta_fn, 'theta', dim=mar) 
ang_idx = np.concatenate((np.arange(was), np.arange(wae+1, prj.shape[0])))

#prj = np.squeeze(reader(prj_fn, 'prj', dim=prj_dim))
prj = reader(prj_fn, 'prj', dim=prj_dim)
-np.log(prj, out=prj)
prj[prj<0] = 0
prj[np.isinf(prj)] = 0
prj[np.isnan(prj)] = 0

recon = np.zeros([prj.shape[1], *obj_dim])
print(f"{prj.shape = }, {prj[ang_idx].shape = }, {theta[ang_idx].shape = }")

### rec data
vol_geom = astra.create_vol_geom(*obj_dim)
prj_geom_f = astra.create_proj_geom('parallel', 1.0, prj.shape[2], theta)
prj_geom_f_cor = astra.functions.geom_postalignment(prj_geom_f, cen)
prj_geom_s = astra.create_proj_geom('parallel', 1.0, prj.shape[2], theta[ang_idx])
prj_geom_s_cor = astra.functions.geom_postalignment(prj_geom_s, cen)
prj_id = astra.create_projector('cuda', prj_geom_f_cor, vol_geom)

sin_f_id = astra.data2d.create('-sino', prj_geom_f_cor, prj[:, 0, :])
sin_f_supp_id = astra.data2d.create('-sino', prj_geom_f_cor, sin_supp)
sf = astra.data2d.get_shared(sin_f_id) 
sf[mar[0]:mar[1]+1] = 0


rec_id = astra.data2d.create('-vol', vol_geom, 0)
#rec_supp_id = astra.data2d.create('-vol', vol_geom, rec_supp)
sin_s_id = astra.data2d.create('-sino', prj_geom_s_cor, prj[:, 0, :][ang_idx])
sin_s_supp_id = astra.data2d.create('-sino', prj_geom_s_cor, sin_supp[ang_idx])

alg_s_cfg = astra.astra_dict('SIRT_CUDA')
alg_s_cfg['ProjectionDataId'] = sin_s_id 
alg_s_cfg['ReconstructionDataId'] = rec_id
alg_s_cfg['option'] = {}
#alg_s_cfg['option']['SinogramMaskId'] = sin_s_supp_id
alg_s_cfg['option']['ReconstructionMaskId'] = rec_supp_id
alg_s_cfg['option']['MaxConstraint'] = 2e-2
#alg_s_cfg['option']['MinConstraint'] = -1e-3
alg_s_id = astra.algorithm.create(alg_s_cfg)

alg_f_cfg = astra.astra_dict('SIRT_CUDA')
alg_f_cfg['ProjectionDataId'] = sin_f_id 
alg_f_cfg['ReconstructionDataId'] = rec_id
alg_f_cfg['option'] = {}
#alg_f_cfg['option']['SinogramMaskId'] = sin_f_supp_id
alg_f_cfg['option']['ReconstructionMaskId'] = rec_supp_id
#alg_f_cfg['option']['MaxConstraint'] = 2e-2
alg_f_cfg['option']['MinConstraint'] = -1e-3
alg_f_id = astra.algorithm.create(alg_f_cfg)

ts = np.zeros([prj.shape[0], prj.shape[2]], dtype=np.float32)
W = astra.OpTomo(prj_id)
for jj in range(prj.shape[1]):
    alg_s_cfg['option'] = {}
    alg_s_cfg['option']['ReconstructionMaskId'] = rec_supp_id
    alg_s_cfg['option']['MaxConstraint'] = 2e-2
    alg_s_id = astra.algorithm.create(alg_s_cfg)
    alg_f_cfg['option'] = {}
    alg_f_cfg['option']['ReconstructionMaskId'] = rec_supp_id
    alg_f_cfg['option']['MinConstraint'] = -1e-3
    alg_f_id = astra.algorithm.create(alg_f_cfg)
    """
    sin_supp_fn = '/run/media/xiao/Data/data/Tiankai/6797/6797_roi_250_mask_415c_sino_mask.tif'
    sin_supp = tifffile.imread(sin_supp_fn)
    ssf = astra.data2d.get_shared(sin_f_supp_id) 
    ssf[:] = sin_supp[:]
    sss = astra.data2d.get_shared(sin_s_supp_id) 
    sss[:] = sin_supp[[ang_idx]]
    """

    prj[:, jj, :] -= ((prj[:, jj, :].sum(axis=1)-np.partition(list(set(prj[:, jj, :].sum(axis=1))), 2)[1]+5)/prj.shape[2])[:, np.newaxis]
    prj[:, jj, :][was:wae+1, :] = 0
    prj[:, jj, :][prj[:, jj, :]<0] = 0

    sf = astra.data2d.get_shared(sin_f_id)  
    sf[:] = prj[:, jj, :]
    ss = astra.data2d.get_shared(sin_s_id) 
    ss[:] = prj[:, jj, :][ang_idx]
    if True:
        for ii in range(100):
            sf = astra.data2d.get_shared(sin_f_id) 
            ssf = astra.data2d.get_shared(sin_f_supp_id) 
            ssf[:] = (sf>0).astype(np.int8)
            sss = astra.data2d.get_shared(sin_s_supp_id) 
            sss[:] = (sf>0).astype(np.int8)[[ang_idx]]

            ss = astra.data2d.get_shared(sin_s_id)    
            median_filter(ss, [1, 3], output=ss)
            astra.algorithm.run(alg_s_id, 10)

            vs = astra.data2d.get_shared(rec_id)
            sf = astra.data2d.get_shared(sin_f_id) 
            ts[:] = W.FP(vs)[:]
            sf[mar[0]:mar[1]+1] = ts[mar[0]:mar[1]+1]
            astra.algorithm.run(alg_f_id, 3)           
            median_filter(sf, [1, 3], output=sf)
            #gaussian_filter(sf[theta_dim[0]:theta_dim[1]+1], 3, output=sf[theta_dim[0]:theta_dim[1]+1])
            astra.algorithm.run(alg_f_id, 3)  

        recon[jj, :] = astra.data2d.get(rec_id)[:]
        rs = astra.data2d.get_shared(rec_supp_id) 
        rs[:] = (gaussian_filter(recon, rec_supp_rel)>rec_thres).astype(np.int8)[:]
        if True:
            sinogram_id, sino = astra.create_sino(recon, prj_id)
            ssf = astra.data2d.get_shared(sin_f_supp_id) 
            ssf[:] = (sino>sin_thres).astype(np.int8)[:]
            ssf[ang_idx] = (prj[ang_idx]>0).astype(np.int8)[:]
            ssf[:] = binary_dilation(ssf, disk(sin_supp_rel))[:]
            sss = astra.data2d.get_shared(sin_s_supp_id) 
            sss[:] = ssf[[ang_idx]]
            astra.data2d.delete(sinogram_id)
    if True:
        astra.data2d.delete(alg_s_id)
        astra.data2d.delete(alg_f_id)
        for ii in range(100):
            sf = astra.data2d.get_shared(sin_f_id) 
            ssf = astra.data2d.get_shared(sin_f_supp_id) 
            ssf[:] = (sf>0).astype(np.int8)
            sss = astra.data2d.get_shared(sin_s_supp_id) 
            sss[:] = (sf>0).astype(np.int8)[[ang_idx]]

            ss = astra.data2d.get_shared(sin_s_id)    
            median_filter(ss, [1, 3], output=ss)
            astra.algorithm.run(alg_s_id, 10)

            vs = astra.data2d.get_shared(rec_id)
            sf = astra.data2d.get_shared(sin_f_id) 
            ts[:] = W.FP(vs)[:]
            sf[mar[0]:mar[1]+1] = ts[mar[0]:mar[1]+1]
            astra.algorithm.run(alg_f_id, 3)           
            median_filter(sf, [1, 3], output=sf)
            #gaussian_filter(sf[theta_dim[0]:theta_dim[1]+1], 3, output=sf[theta_dim[0]:theta_dim[1]+1])
            astra.algorithm.run(alg_f_id, 3)  

        recon[jj, :] = astra.data2d.get(rec_id)[:]
        rs = astra.data2d.get_shared(rec_supp_id) 
        rs[:] = (gaussian_filter(recon, rec_supp_rel)>rec_thres).astype(np.int8)[:]
        if True:
            sinogram_id, sino = astra.create_sino(recon, prj_id)
            ssf = astra.data2d.get_shared(sin_f_supp_id) 
            ssf[:] = (sino>sin_thres).astype(np.int8)[:]
            ssf[ang_idx] = (prj[ang_idx]>0).astype(np.int8)[:]
            ssf[:] = binary_dilation(ssf, disk(sin_supp_rel))[:]
            sss = astra.data2d.get_shared(sin_s_supp_id) 
            sss[:] = ssf[[ang_idx]]
            astra.data2d.delete(sinogram_id)
            
    recon[jj, :] = astra.data2d.get(rec_id)[:]
    print(f"{jj}th slice recon finishes at {time.asctime()}")

"""
fp_geom = astra.create_proj_geom('parallel', 1., prj.shape[1], theta)
fp_id = astra.create_projector('cuda', fp_geom, vol_geom)
sinogram_id, sinogram = astra.create_sino(recon, fp_id)
print(f"    forward projection finishes at {time.asctime()}")
"""

"""
ss = astra.data2d.get_shared(sin_s_id)    
ss[:] = median_filter(ss, [1, 5])
tifffile.imsave('/media/xiao_usb2/Tiankai/66881_1080_391x590_sino_s.tif', ss.astype(np.float32))

sf = astra.data2d.get_shared(sin_f_id)    
sf[:] = median_filter(sf, [1, 5])
tifffile.imsave('/media/xiao_usb2/Tiankai/66881_1080_391x590_sino_f.tif', sf.astype(np.float32))
"""


astra.algorithm.delete(alg_s_id)
astra.data2d.delete(sin_s_id)
astra.data2d.delete(sin_s_supp_id)

astra.algorithm.delete(alg_f_id)
astra.data2d.delete(sin_f_id)
astra.data2d.delete(sin_f_supp_id)

astra.data2d.delete(rec_id)
astra.data2d.delete(rec_supp_id)

astra.data2d.delete(sinogram_id)
astra.projector.delete(fp_id)
astra.projector.delete(prj_id)
astra.functions.clear()


In [46]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Dec 12 12:18:13 2020

@author: xiao
"""

### prepare data
sin_thres = 0.15
sin_supp_rel = 10
rec_thres = 2.5e-4
rec_supp_rel = 15

#prj_dim = np.s_[:, 250:251, 123:677]
prj_dim = np.s_[:, 200:300, 123:580]
obj_dim = [415, 415]

cen = 0
print(f'{cen = }')
prj_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_6797.h5' 
theta_fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_6797.h5' 
obj_fn = '' 
with h5py.File(theta_fn, 'r') as f:
    mar = f['missing_ang_range'][:]
    
rec_supp_fn = '/run/media/xiao/Data/data/Tiankai/6797/6797_roi_250_mask_415b_recon_mask.tif'
rec_supp = tifffile.imread(rec_supp_fn)
rec_supp = binary_dilation(rec_supp, disk(rec_supp_rel))
sin_supp_fn = '/run/media/xiao/Data/data/Tiankai/6797/6797_roi_250_mask_415c_sino_mask.tif'
sin_supp = tifffile.imread(sin_supp_fn)
sin_supp = binary_dilation(sin_supp, disk(sin_supp_rel))

theta, was, wae = reader(theta_fn, 'theta', dim=mar) 
ang_idx = np.concatenate((np.arange(was), np.arange(wae+1, prj.shape[0])))

#prj = np.squeeze(reader(prj_fn, 'prj', dim=prj_dim))
prj = reader(prj_fn, 'prj', dim=prj_dim)
-np.log(prj, out=prj)
prj[prj<0] = 0
prj[np.isinf(prj)] = 0
prj[np.isnan(prj)] = 0
median_filter(prj, size=[1, 5, 5], output=prj)

recon = np.zeros([prj.shape[1], *obj_dim])
tr = np.zeros(obj_dim)
print(f"{prj.shape = }, {prj[ang_idx].shape = }, {theta[ang_idx].shape = }")

### rec data
vol_geom = astra.create_vol_geom(*obj_dim)
prj_geom_f = astra.create_proj_geom('parallel', 1.0, prj.shape[2], theta)
prj_geom_f_cor = astra.functions.geom_postalignment(prj_geom_f, cen)
prj_geom_s = astra.create_proj_geom('parallel', 1.0, prj.shape[2], theta[ang_idx])
prj_geom_s_cor = astra.functions.geom_postalignment(prj_geom_s, cen)
prj_id = astra.create_projector('cuda', prj_geom_f_cor, vol_geom)

sin_f_id = astra.data2d.create('-sino', prj_geom_f_cor, prj[:, 0, :])
sin_f_supp_id = astra.data2d.create('-sino', prj_geom_f_cor, sin_supp)
sf = astra.data2d.get_shared(sin_f_id) 
sf[mar[0]:mar[1]+1] = 0


rec_id = astra.data2d.create('-vol', vol_geom, 0)
rec_supp_id = astra.data2d.create('-vol', vol_geom, rec_supp)
sin_s_id = astra.data2d.create('-sino', prj_geom_s_cor, prj[:, 0, :][ang_idx])
sin_s_supp_id = astra.data2d.create('-sino', prj_geom_s_cor, sin_supp[ang_idx])

alg_s_cfg = astra.astra_dict('SIRT_CUDA')
alg_s_cfg['ProjectionDataId'] = sin_s_id 
alg_s_cfg['ReconstructionDataId'] = rec_id
# alg_s_cfg['option'] = {}
#alg_s_cfg['option']['SinogramMaskId'] = sin_s_supp_id
# alg_s_cfg['option']['ReconstructionMaskId'] = rec_supp_id
# alg_s_cfg['option']['MaxConstraint'] = 2e-2
#alg_s_cfg['option']['MinConstraint'] = -1e-3
# alg_s_id = astra.algorithm.create(alg_s_cfg)

alg_f_cfg = astra.astra_dict('SIRT_CUDA')
alg_f_cfg['ProjectionDataId'] = sin_f_id 
alg_f_cfg['ReconstructionDataId'] = rec_id
# alg_f_cfg['option'] = {}
#alg_f_cfg['option']['SinogramMaskId'] = sin_f_supp_id
#alg_f_cfg['option']['ReconstructionMaskId'] = rec_supp_id
#alg_f_cfg['option']['MaxConstraint'] = 2e-2
# alg_f_cfg['option']['MinConstraint'] = -1e-3
# alg_f_id = astra.algorithm.create(alg_f_cfg)

ts = np.zeros([prj.shape[0], prj.shape[2]], dtype=np.float32)
W = astra.OpTomo(prj_id)
for jj in range(prj.shape[1]):
    """
    sin_supp_fn = '/run/media/xiao/Data/data/Tiankai/6797/6797_roi_250_mask_415c_sino_mask.tif'
    sin_supp = tifffile.imread(sin_supp_fn)
    ssf = astra.data2d.get_shared(sin_f_supp_id) 
    ssf[:] = sin_supp[:]
    sss = astra.data2d.get_shared(sin_s_supp_id) 
    sss[:] = sin_supp[[ang_idx]]
    """

    prj[:, jj, :] -= ((prj[:, jj, :].sum(axis=1)-np.partition(list(set(prj[:, jj, :].sum(axis=1))), 2)[1]+5)/prj.shape[2])[:, np.newaxis]
    prj[:, jj, :][was:wae+1, :] = 0
    prj[:, jj, :][prj[:, jj, :]<0] = 0

    sf = astra.data2d.get_shared(sin_f_id)  
    sf[:] = prj[:, jj, :]
    ss = astra.data2d.get_shared(sin_s_id) 
    ss[:] = prj[:, jj, :][ang_idx]
    
    astra.data2d.get_shared(rec_id)[:] = tr[:]
    #tr = astra.data2d.get(rec_id)
    #tr[:] = np.zeros(obj_dim)
    if True:
        alg_s_cfg['option'] = {}
        alg_s_cfg['option']['ReconstructionMaskId'] = rec_supp_id
        alg_s_cfg['option']['MaxConstraint'] = 2e-2
        alg_s_cfg['option']['MinConstraint'] = -1e-4
        alg_s_id = astra.algorithm.create(alg_s_cfg)
        
        alg_f_cfg['option'] = {}
        alg_f_cfg['option']['ReconstructionMaskId'] = rec_supp_id
        alg_f_cfg['option']['MaxConstraint'] = 2e-2
        alg_f_cfg['option']['MinConstraint'] = -1e-4
        alg_f_id = astra.algorithm.create(alg_f_cfg)
        for ii in range(50):
            ss = astra.data2d.get_shared(sin_s_id)    
            median_filter(ss, [1, 3], output=ss)
            astra.algorithm.run(alg_s_id, 10)

            vs = astra.data2d.get_shared(rec_id)
            sf = astra.data2d.get_shared(sin_f_id) 
            ts[:] = W.FP(vs)[:]
            sf[mar[0]:mar[1]+1] = ts[mar[0]:mar[1]+1]
            astra.algorithm.run(alg_f_id, 3)           
            median_filter(sf, [1, 3], output=sf)
            astra.algorithm.run(alg_f_id, 3)  

        recon[jj, :] = astra.data2d.get(rec_id)[:]
        rs = astra.data2d.get_shared(rec_supp_id) 
        rs[:] = (gaussian_filter(recon[jj], rec_supp_rel)>rec_thres).astype(np.int8)[:]
        if False:
            sinogram_id, sino = astra.create_sino(recon[jj], prj_id)
            ssf = astra.data2d.get_shared(sin_f_supp_id) 
            ssf[:] = (sino>sin_thres).astype(np.int8)[:]
            ssf[ang_idx] = (prj[ang_idx, jj, :]>0).astype(np.int8)[:]
            ssf[:] = binary_dilation(ssf, disk(sin_supp_rel))[:]
            sss = astra.data2d.get_shared(sin_s_supp_id) 
            sss[:] = ssf[[ang_idx]]
            astra.data2d.delete(sinogram_id)
        astra.data2d.delete(alg_s_id)
        astra.data2d.delete(alg_f_id)
    if True:
        alg_s_cfg['option'] = {}
        #alg_s_cfg['option']['SinogramMaskId'] = sin_s_supp_id
        alg_s_cfg['option']['ReconstructionMaskId'] = rec_supp_id
        alg_s_cfg['option']['MaxConstraint'] = 2e-2
        #alg_s_cfg['option']['MinConstraint'] = -1e-3
        alg_s_id = astra.algorithm.create(alg_s_cfg)
        
        alg_f_cfg['option'] = {}
        #alg_f_cfg['option']['SinogramMaskId'] = sin_f_supp_id
        alg_f_cfg['option']['ReconstructionMaskId'] = rec_supp_id
        #alg_f_cfg['option']['MaxConstraint'] = 2e-2
        alg_f_cfg['option']['MinConstraint'] = -1e-4
        alg_f_id = astra.algorithm.create(alg_f_cfg)
        
        for ii in range(50):
            ss = astra.data2d.get_shared(sin_s_id)    
            median_filter(ss, [1, 3], output=ss)
            astra.algorithm.run(alg_s_id, 10)

            vs = astra.data2d.get_shared(rec_id)
            sf = astra.data2d.get_shared(sin_f_id) 
            ts[:] = W.FP(vs)[:]
            sf[mar[0]:mar[1]+1] = ts[mar[0]:mar[1]+1]
            astra.algorithm.run(alg_f_id, 3)           
            median_filter(sf, [1, 3], output=sf)
            astra.algorithm.run(alg_f_id, 3)  
        astra.data2d.delete(alg_s_id)
        astra.data2d.delete(alg_f_id)
            
    recon[jj, :] = astra.data2d.get(rec_id)[:]
    print(f"{jj}th slice recon finishes at {time.asctime()}")

"""
fp_geom = astra.create_proj_geom('parallel', 1., prj.shape[1], theta)
fp_id = astra.create_projector('cuda', fp_geom, vol_geom)
sinogram_id, sinogram = astra.create_sino(recon, fp_id)
print(f"    forward projection finishes at {time.asctime()}")
"""

"""
ss = astra.data2d.get_shared(sin_s_id)    
ss[:] = median_filter(ss, [1, 5])
tifffile.imsave('/media/xiao_usb2/Tiankai/66881_1080_391x590_sino_s.tif', ss.astype(np.float32))

sf = astra.data2d.get_shared(sin_f_id)    
sf[:] = median_filter(sf, [1, 5])
tifffile.imsave('/media/xiao_usb2/Tiankai/66881_1080_391x590_sino_f.tif', sf.astype(np.float32))
"""

if True:
    # astra.algorithm.delete(alg_s_id)
    astra.data2d.delete(sin_s_id)
    astra.data2d.delete(sin_s_supp_id)

    # astra.algorithm.delete(alg_f_id)
    astra.data2d.delete(sin_f_id)
    astra.data2d.delete(sin_f_supp_id)

    astra.data2d.delete(rec_id)
    astra.data2d.delete(rec_supp_id)

    astra.data2d.delete(sinogram_id)
    #astra.projector.delete(fp_id)
    astra.projector.delete(prj_id)
    astra.functions.clear()


cen = 0


<ipython-input-2-4b71a91cd265>:16: RuntimeWarning: divide by zero encountered in log
  tem = -np.log((f['/img_tomo'][dim] - f['/img_dark_avg'][dim])/(f['/img_bkg_avg'][dim] - f['/img_dark_avg'][dim]))
<ipython-input-46-dc8de75f32d9>:39: RuntimeWarning: divide by zero encountered in log
  -np.log(prj, out=prj)
<ipython-input-46-dc8de75f32d9>:39: RuntimeWarning: invalid value encountered in log
  -np.log(prj, out=prj)


prj.shape = (386, 100, 457), prj[ang_idx].shape = (281, 100, 457), theta[ang_idx].shape = (281,)
0th slice recon finishes at Sat Dec 12 17:49:53 2020
1th slice recon finishes at Sat Dec 12 17:49:56 2020
2th slice recon finishes at Sat Dec 12 17:49:59 2020
3th slice recon finishes at Sat Dec 12 17:50:02 2020
4th slice recon finishes at Sat Dec 12 17:50:05 2020
5th slice recon finishes at Sat Dec 12 17:50:08 2020
6th slice recon finishes at Sat Dec 12 17:50:11 2020
7th slice recon finishes at Sat Dec 12 17:50:14 2020
8th slice recon finishes at Sat Dec 12 17:50:17 2020
9th slice recon finishes at Sat Dec 12 17:50:20 2020
10th slice recon finishes at Sat Dec 12 17:50:23 2020
11th slice recon finishes at Sat Dec 12 17:50:26 2020
12th slice recon finishes at Sat Dec 12 17:50:29 2020
13th slice recon finishes at Sat Dec 12 17:50:32 2020
14th slice recon finishes at Sat Dec 12 17:50:36 2020
15th slice recon finishes at Sat Dec 12 17:50:39 2020
16th slice recon finishes at Sat Dec 12 17:50:42 

In [45]:
plt.figure(200)
plt.imshow(astra.data2d.get_shared(rec_id)[:])

ValueError: Data object not found

In [43]:
plt.figure(10)
plt.imshow(recon[0])

plt.figure(11)
plt.imshow(recon[-1])

In [ ]:
print(np.partition(list(set(prj.sum(axis=1))), 2)[1])

In [ ]:
print(vs.shape, ts.shape)

In [47]:
tifffile.imsave('/run/media/xiao/Data/data/Tiankai/6797/6797_roi_200-300_recon.tif', recon.astype(np.float32))